In [1]:
def iter_by_order(order, dim, extended=True):
    """
    Iterates over all combinations of basis functions indexes
    needed to create multidimensional basis in a way that creates hierarchical basis
    :param order: desired order of multidimensional basis
    :param dim: dimension of the basis
    :param extended

    :yields: tuple containing indexes, use in _combine_polyvals and _combine_polyvals_der
    :return: None
    """

    # nth(iter(map(lambda x: x + (order - reduce(add,x),)), range(order)), dim)
    # nth(dim, iterate(map(lambda x: x + (order - reduce(add,x),)), map(tuple, range(order))))
    # nth(2, iterate(map(lambda x: x + (order - reduce(add,x),)), map(lambda x: (x,), range(order))))
    porder = order + 1
    if dim == 1:
        for i in range(porder):
            yield i,
        return
    elif dim == 2:
        for k in range(porder):
            for i in range(k + 1):
                yield i, k - i
        if not extended: return
        for k in range(1, porder):
            for i in range(1, porder):
                if i + k <= porder - 1:
                    continue
                yield i, k

    elif dim == 3:
        for k in range(porder):
            for j in range(k + 1):
                for i in range(j + 1):
                    yield i, j - i, k - j
        return


In [2]:
def gramm_schmidt(base, dot_prod):
    '''
    Ortogonalizuje bázi base vůči skálárnímu součinu dot_prod
    :param base:
    :param dot_prod:
    :return:
    '''
    new_base = []
    for i in range(len(base)):
        new_base.append(ortogonalize(base[i], new_base, dot_prod))
    return new_base


def ortogonalize(vec, base, dot_prod):
    for j in range(len(base)):
        kap = - dot_prod(vec, base[j]) / dot_prod(base[j], base[j])
        vec = vec + kap * base[j]
    return vec

def dot_prods_matrix(basis, dot_prod):
    Np = len(basis)
    from sympy.matrices import ones
    vendM = ones(Np, Np)
    for i in range(Np):
        for j in range(Np):
            vendM[i, j] = dot_prod(basis[i], basis[j])
    return vendM

In [3]:
square_dots = [lambda f, g: integrate(f*g, (x, 0, 1)),
                lambda f, g: integrate(integrate(f*g, (y, 0, 1)),(x, 0, 1)),
                lambda f, g: integrate(integrate(integrate(f*g, (z, 0, 1)),(y, 0, 1)),(x, 0, 1))
               ]# Fubini, yayy!

In [6]:
x, y = var(("x", "y"))
r, s = var(("r", "s"))
order = 2
D = 2

# Legendre basis on $[0, 1]\times[0, 1]$

In [9]:
tensorP = []
for m, idx in enumerate(iter_by_order(order, 2)):
    print("P_{} = {}".format(m, expand(jacobi_P(idx[0], 0, 0, x)*
                                       jacobi_P(idx[1], 0, 0, y))))
    tensorP.append(jacobi_P(idx[0], 0, 0, x)*jacobi_P(idx[1], 0,0,y))

P_0 = 1
P_1 = y
P_2 = x
P_3 = 3/2*y^2 - 1/2
P_4 = x*y
P_5 = 3/2*x^2 - 1/2
P_6 = 3/2*x^2*y - 1/2*y
P_7 = 3/2*x*y^2 - 1/2*x
P_8 = 9/4*x^2*y^2 - 3/4*x^2 - 3/4*y^2 + 1/4


Legendre basis is not ortogonal

In [10]:
dot_prods_matrix(tensorP, square_dots[D - 1])

Matrix([
[  1,  1/2,  1/2,    0,  1/4,    0,    0,    0,    0],
[1/2,  1/3,  1/4,  1/8,  1/6,    0,    0, 1/16,    0],
[1/2,  1/4,  1/3,    0,  1/6,  1/8, 1/16,    0,    0],
[  0,  1/8,    0,  1/5, 1/16,    0,    0, 1/10,    0],
[1/4,  1/6,  1/6, 1/16,  1/9, 1/16, 1/24, 1/24, 1/64],
[  0,    0,  1/8,    0, 1/16,  1/5, 1/10,    0,    0],
[  0,    0, 1/16,    0, 1/24, 1/10, 1/15, 1/64, 1/40],
[  0, 1/16,    0, 1/10, 1/24,    0, 1/64, 1/15, 1/40],
[  0,    0,    0,    0, 1/64,    0, 1/40, 1/40, 1/25]])

By gramm-shimdt process we get ortogonal basis. However its "design" matrix is badly conditioned.

In [11]:
tensorP_ort = gramm_schmidt(tensorP, square_dots[D - 1])
tensorP_ort

[1,
 y - 1/2,
 x - 1/2,
 3/2*y^2 - 3/2*y + 1/4,
 x*y - 1/2*x - 1/2*y + 1/4,
 3/2*x^2 - 3/2*x + 1/4,
 -3/4*x^2 + 1/2*(3*x^2 - 1)*y - 3/2*x*y + 3/4*x + 3/4*y - 1/8,
 1/2*(3*y^2 - 1)*x - 3/2*x*y - 3/4*y^2 + 3/4*x + 3/4*y - 1/8,
 1/4*(3*x^2 - 1)*(3*y^2 - 1) - 3/4*(3*y^2 - 1)*x + 9/8*x^2 - 3/4*(3*x^2 - 1)*y + 9/4*x*y + 9/8*y^2 - 9/8*x - 9/8*y - 3/16]

In [12]:
dot_prods_matrix(tensorP_ort, square_dots[D - 1])

Matrix([
[1,    0,    0,    0,     0,    0,     0,     0,      0],
[0, 1/12,    0,    0,     0,    0,     0,     0,      0],
[0,    0, 1/12,    0,     0,    0,     0,     0,      0],
[0,    0,    0, 1/80,     0,    0,     0,     0,      0],
[0,    0,    0,    0, 1/144,    0,     0,     0,      0],
[0,    0,    0,    0,     0, 1/80,     0,     0,      0],
[0,    0,    0,    0,     0,    0, 1/960,     0,      0],
[0,    0,    0,    0,     0,    0,     0, 1/960,      0],
[0,    0,    0,    0,     0,    0,     0,     0, 1/6400]])

# Legendre basis with tansformation
Tansforming to element $[-1, 1] \times [-1 , 1]$ by substitution $r = 2x - 1$ and $s = 2y - 1$

In [13]:
tensorP = []
for m, idx in enumerate(iter_by_order(order, D)):
    jpol = (jacobi_P(idx[0], 0, 0, r)*
            jacobi_P(idx[1], 0, 0, s)).subs({r : 2*x - 1, s : 2*y - 1})
    print("P_{} = {}".format(m, expand(jpol)))
    tensorP.append(jpol)

P_0 = 1
P_1 = 2*y - 1
P_2 = 2*x - 1
P_3 = 6*y^2 - 6*y + 1
P_4 = 4*x*y - 2*x - 2*y + 1
P_5 = 6*x^2 - 6*x + 1
P_6 = 12*x^2*y - 6*x^2 - 12*x*y + 6*x + 2*y - 1
P_7 = 12*x*y^2 - 12*x*y - 6*y^2 + 2*x + 6*y - 1
P_8 = 36*x^2*y^2 - 36*x^2*y - 36*x*y^2 + 6*x^2 + 36*x*y + 6*y^2 - 6*x - 6*y + 1


In [14]:
dot_prods_matrix(tensorP, square_dots[D - 1])

Matrix([
[1,   0,   0,   0,   0,   0,    0,    0,    0],
[0, 1/3,   0,   0,   0,   0,    0,    0,    0],
[0,   0, 1/3,   0,   0,   0,    0,    0,    0],
[0,   0,   0, 1/5,   0,   0,    0,    0,    0],
[0,   0,   0,   0, 1/9,   0,    0,    0,    0],
[0,   0,   0,   0,   0, 1/5,    0,    0,    0],
[0,   0,   0,   0,   0,   0, 1/15,    0,    0],
[0,   0,   0,   0,   0,   0,    0, 1/15,    0],
[0,   0,   0,   0,   0,   0,    0,    0, 1/25]])

In [15]:
_.condition_number()

25

In [16]:
# In case we get sfepy imports working
# geometry = Struct(n_vertex=2,
#                   dim=1,
#                   coors=nm.array([0, 1]))
# order = 4
# ts = LegendreTensorProductPolySpace('legb', geometry, order)
# ls = LegendrePolySpace('legb', geometry, order)

# Legendre tensor basis gradients

In [160]:
tensorP_grad = []
tensorP_ngrad = []
tensorP_ngradf = []
import numpy as nm
for i, P in enumerate(tensorP):
    gradxP(x,y) = diff(P, x)
    gradyP(x,y) = diff(P, y)
    gradP(x, y) = [gradxP, gradyP]
    print(str(gradP(x,y)))
    tensorP_grad.append(gradP)
    ngradPs = ()
    tensorP_ngradf.append((fast_callable(diff(P, x), vars = (x,y)), 
                           fast_callable(diff(P, y), vars = (x,y))))
    sgradxP = str(gradxP(x,y)).replace("^", "**")
    sgradyP = str(gradyP(x,y)).replace("^", "**")
    tensorP_ngrad.append(eval("lambda x, y: nm.stack([{s1}*nm.ones(nm.shape(x)), \
                                                      {s2}*nm.ones(nm.shape(y))])"
                              .format(s1=sgradxP, s2=sgradyP)))

(0, 0)
(0, 2)
(2, 0)
(0, 12*y - 6)
(4*y - 2, 4*x - 2)
(12*x - 6, 0)
(6*(2*x - 1)*(2*y - 1), 3*(2*x - 1)^2 - 1)
(3*(2*y - 1)^2 - 1, 6*(2*x - 1)*(2*y - 1))
(3*(3*(2*y - 1)^2 - 1)*(2*x - 1), 3*(3*(2*x - 1)^2 - 1)*(2*y - 1))


In [175]:
X, Y = nm.meshgrid(nm.linspace(0,1, 10), nm.linspace(0,1, 10))

gradZ = [p(X, Y) for p in tensorP_ngrad]

   

In [179]:
gradZ[5][0][:, 0]

array([-6., -6., -6., -6., -6., -6., -6., -6., -6., -6.])

In [181]:
X[: , 0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])